# UCITAVANJE PODATAKA

In [4]:
import numpy as np

In [5]:
def load_dataset(path):
    data = np.load(path)
    X_train = data['X_train']
    y_train = data['y_train']
    X_val = data['X_val']
    y_val = data['y_val']
    X_test = data['X_test']
    y_test = data['y_test']
    print(f"Dataset loaded from {path}")
    return X_train, y_train, X_val, y_val, X_test, y_test

In [6]:
X_train, y_train, X_val, y_val, X_test, y_test = load_dataset('my_dataset.npz')

Dataset loaded from my_dataset.npz


# SEED

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout, BatchNormalization
import tensorflow as tf
import os

import random
os.environ['PYTHONHASHSEED'] = '0'
random.seed(0)
np.random.seed(0)
import tensorflow as tf
tf.random.set_seed(0)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

In [51]:
from tensorflow.keras.callbacks import EarlyStopping
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [84]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import TensorBoard

# LOSO - unakrsna validacija

In [110]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Bidirectional

def create_model(input_shape, num_classes):
    model = Sequential()
    model.add(Bidirectional(LSTM(128, return_sequences=True, activation='tanh'), input_shape=input_shape))
    model.add(Dropout(0.3))
    model.add(LSTM(64, return_sequences=False, activation='tanh'))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(32, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])
    return model

In [111]:
import os
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

DATA_PATH = os.path.join('Master_Data - Copy')
actions = np.array(['Ja','Mogu','Riba','Sve','Momak','Devojka','Opasno','Kada','Zasto','Hvala'])
sequence_length = 35
label_map = {label:num for num, label in enumerate(actions)}

genders = ['man', 'woman']
persons = {
    'man': ['person2', 'person4'],
    'woman': ['person1', 'person3', 'person5']
}


all_people = []
for gender in genders:
    for person in persons[gender]:
        all_people.append((gender, person))

results = []

for i_test, (test_gender, test_person) in enumerate(all_people):
    remaining = all_people[:i_test] + all_people[i_test+1:]
    for i_val, (val_gender, val_person) in enumerate(remaining):
        train_people = remaining[:i_val] + remaining[i_val+1:]

        sequences_train, labels_train = [], []
        sequences_val, labels_val = [], []
        sequences_test, labels_test = [], []

        for gender in genders:
            for person in persons[gender]:
                for action in actions:
                    action_path = os.path.join(DATA_PATH, gender, person, action)
                    if not os.path.exists(action_path):
                        continue
                    for sequence in np.array(os.listdir(action_path)).astype(int):
                        window = []
                        for frame_num in range(sequence_length):
                            res = np.load(os.path.join(action_path, str(sequence), f"{frame_num}.npy"))
                            window.append(res)

                        if (gender, person) in train_people:
                            sequences_train.append(window)
                            labels_train.append(label_map[action])
                        elif (gender, person) == (val_gender, val_person):
                            sequences_val.append(window)
                            labels_val.append(label_map[action])
                        elif (gender, person) == (test_gender, test_person):
                            sequences_test.append(window)
                            labels_test.append(label_map[action])

        X_train = np.array(sequences_train)
        y_train = to_categorical(labels_train).astype(np.float32)
        X_val = np.array(sequences_val)
        y_val = to_categorical(labels_val).astype(np.float32)
        X_test = np.array(sequences_test)
        y_test = to_categorical(labels_test).astype(np.float32)

        model = create_model(input_shape=(sequence_length, X_train.shape[2]), num_classes=y_train.shape[1])

        history = model.fit(
            X_train, y_train,
            epochs=100,
            batch_size=64,
            validation_data=(X_val, y_val),
            callbacks=[early_stop],
            verbose=1
        )

        test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
        print(f"Test accuracy (test={test_person}, val={val_person}): {test_acc:.4f}")

        results.append({
            'test_person': test_person,
            'val_person': val_person,
            'test_accuracy': test_acc,
            'test_loss': test_loss,
            'train_history': history.history
        })

Epoch 1/100
10/10 [==============================] - 10s 377ms/step - loss: 2.3187 - categorical_accuracy: 0.1000 - val_loss: 2.3025 - val_categorical_accuracy: 0.1000
Epoch 2/100
10/10 [==============================] - 3s 258ms/step - loss: 2.3105 - categorical_accuracy: 0.0850 - val_loss: 2.3044 - val_categorical_accuracy: 0.1400
Epoch 3/100
10/10 [==============================] - 2s 246ms/step - loss: 2.3066 - categorical_accuracy: 0.0900 - val_loss: 2.3020 - val_categorical_accuracy: 0.1000
Epoch 4/100
10/10 [==============================] - 2s 253ms/step - loss: 2.3033 - categorical_accuracy: 0.1117 - val_loss: 2.2946 - val_categorical_accuracy: 0.1000
Epoch 5/100
10/10 [==============================] - 2s 235ms/step - loss: 2.2902 - categorical_accuracy: 0.1450 - val_loss: 2.2643 - val_categorical_accuracy: 0.2050
Epoch 6/100
10/10 [==============================] - 2s 239ms/step - loss: 2.2783 - categorical_accuracy: 0.1517 - val_loss: 2.2156 - val_categorical_accuracy: 0.19

Epoch 50/100
10/10 [==============================] - 4s 386ms/step - loss: 1.5113 - categorical_accuracy: 0.3883 - val_loss: 1.7747 - val_categorical_accuracy: 0.2700
Epoch 51/100
10/10 [==============================] - 3s 346ms/step - loss: 1.4105 - categorical_accuracy: 0.3900 - val_loss: 1.8375 - val_categorical_accuracy: 0.2850
Epoch 52/100
10/10 [==============================] - 3s 317ms/step - loss: 1.3628 - categorical_accuracy: 0.4483 - val_loss: 1.8597 - val_categorical_accuracy: 0.3050
Epoch 53/100
10/10 [==============================] - 3s 304ms/step - loss: 1.4351 - categorical_accuracy: 0.3983 - val_loss: 1.8375 - val_categorical_accuracy: 0.2550
Epoch 54/100
10/10 [==============================] - 3s 284ms/step - loss: 1.3747 - categorical_accuracy: 0.4517 - val_loss: 1.7831 - val_categorical_accuracy: 0.3050
Epoch 55/100
10/10 [==============================] - 3s 296ms/step - loss: 1.3629 - categorical_accuracy: 0.4417 - val_loss: 1.8480 - val_categorical_accuracy:

Epoch 99/100
10/10 [==============================] - 3s 304ms/step - loss: 0.5875 - categorical_accuracy: 0.7583 - val_loss: 1.7829 - val_categorical_accuracy: 0.5000
Epoch 100/100
10/10 [==============================] - 3s 314ms/step - loss: 0.5809 - categorical_accuracy: 0.7600 - val_loss: 2.0809 - val_categorical_accuracy: 0.4000
Test accuracy (test=person2, val=person4): 0.4500
Epoch 1/100
10/10 [==============================] - 14s 487ms/step - loss: 2.3283 - categorical_accuracy: 0.0817 - val_loss: 2.3056 - val_categorical_accuracy: 0.1000
Epoch 2/100
10/10 [==============================] - 3s 302ms/step - loss: 2.3115 - categorical_accuracy: 0.1017 - val_loss: 2.3046 - val_categorical_accuracy: 0.1000
Epoch 3/100
10/10 [==============================] - 3s 292ms/step - loss: 2.3130 - categorical_accuracy: 0.0967 - val_loss: 2.3038 - val_categorical_accuracy: 0.1000
Epoch 4/100
10/10 [==============================] - 3s 294ms/step - loss: 2.3009 - categorical_accuracy: 0.108

Epoch 64/100
10/10 [==============================] - 5s 532ms/step - loss: 1.4424 - categorical_accuracy: 0.3817 - val_loss: 1.5541 - val_categorical_accuracy: 0.3600
Epoch 65/100
10/10 [==============================] - 5s 509ms/step - loss: 1.4403 - categorical_accuracy: 0.3983 - val_loss: 1.4702 - val_categorical_accuracy: 0.4100
Epoch 66/100
10/10 [==============================] - 5s 519ms/step - loss: 1.3814 - categorical_accuracy: 0.4617 - val_loss: 1.6743 - val_categorical_accuracy: 0.3700
Epoch 67/100
10/10 [==============================] - 5s 526ms/step - loss: 1.4282 - categorical_accuracy: 0.4433 - val_loss: 1.7265 - val_categorical_accuracy: 0.3500
Epoch 68/100
10/10 [==============================] - 5s 500ms/step - loss: 1.4251 - categorical_accuracy: 0.4200 - val_loss: 1.6146 - val_categorical_accuracy: 0.4250
Epoch 69/100
10/10 [==============================] - 5s 523ms/step - loss: 1.3929 - categorical_accuracy: 0.4483 - val_loss: 1.9618 - val_categorical_accuracy:

Epoch 61/100
10/10 [==============================] - 6s 587ms/step - loss: 1.2204 - categorical_accuracy: 0.5050 - val_loss: 1.6024 - val_categorical_accuracy: 0.3250
Epoch 62/100
10/10 [==============================] - 6s 563ms/step - loss: 1.1838 - categorical_accuracy: 0.5617 - val_loss: 1.6342 - val_categorical_accuracy: 0.3350
Epoch 63/100
10/10 [==============================] - 6s 563ms/step - loss: 1.2478 - categorical_accuracy: 0.5067 - val_loss: 1.8826 - val_categorical_accuracy: 0.2600
Epoch 64/100
10/10 [==============================] - 5s 551ms/step - loss: 1.3853 - categorical_accuracy: 0.4233 - val_loss: 1.7636 - val_categorical_accuracy: 0.3200
Epoch 65/100
10/10 [==============================] - 5s 544ms/step - loss: 1.2877 - categorical_accuracy: 0.4783 - val_loss: 1.5003 - val_categorical_accuracy: 0.4400
Epoch 66/100
10/10 [==============================] - 6s 599ms/step - loss: 1.1804 - categorical_accuracy: 0.5417 - val_loss: 1.7292 - val_categorical_accuracy:

...

10/10 [==============================] - 25s 2s/step - loss: 1.1336 - categorical_accuracy: 0.5383 - val_loss: 1.5050 - val_categorical_accuracy: 0.4100
Epoch 80/100
10/10 [==============================] - 23s 2s/step - loss: 1.0506 - categorical_accuracy: 0.5450 - val_loss: 1.6380 - val_categorical_accuracy: 0.4100
Epoch 81/100
10/10 [==============================] - 24s 2s/step - loss: 1.0151 - categorical_accuracy: 0.5417 - val_loss: 1.6022 - val_categorical_accuracy: 0.4250
Epoch 82/100
10/10 [==============================] - 26s 3s/step - loss: 1.0624 - categorical_accuracy: 0.5283 - val_loss: 1.7220 - val_categorical_accuracy: 0.4250
Epoch 83/100
10/10 [==============================] - 24s 2s/step - loss: 0.9982 - categorical_accuracy: 0.5850 - val_loss: 1.6344 - val_categorical_accuracy: 0.4350
Epoch 84/100
10/10 [==============================] - 27s 3s/step - loss: 0.9951 - categorical_accuracy: 0.5733 - val_loss: 1.9385 - val_categorical_accuracy: 0.3900
Test accuracy (te

Epoch 12/100
10/10 [==============================] - 28s 3s/step - loss: 1.8206 - categorical_accuracy: 0.2483 - val_loss: 1.8141 - val_categorical_accuracy: 0.2800
Epoch 13/100
10/10 [==============================] - 25s 3s/step - loss: 1.7850 - categorical_accuracy: 0.2617 - val_loss: 1.7083 - val_categorical_accuracy: 0.2650
Epoch 14/100
10/10 [==============================] - 25s 3s/step - loss: 1.7474 - categorical_accuracy: 0.2567 - val_loss: 1.7194 - val_categorical_accuracy: 0.3650
Epoch 15/100
10/10 [==============================] - 23s 2s/step - loss: 1.7359 - categorical_accuracy: 0.2583 - val_loss: 1.7079 - val_categorical_accuracy: 0.2600
Epoch 16/100
10/10 [==============================] - 24s 2s/step - loss: 1.7374 - categorical_accuracy: 0.2450 - val_loss: 1.7058 - val_categorical_accuracy: 0.3300
Epoch 17/100
10/10 [==============================] - 23s 2s/step - loss: 1.6796 - categorical_accuracy: 0.3050 - val_loss: 1.6604 - val_categorical_accuracy: 0.3400
Epoc

10/10 [==============================] - 25s 3s/step - loss: 1.3610 - categorical_accuracy: 0.4500 - val_loss: 1.6566 - val_categorical_accuracy: 0.3450
Epoch 70/100
10/10 [==============================] - 25s 3s/step - loss: 1.4229 - categorical_accuracy: 0.4317 - val_loss: 1.5078 - val_categorical_accuracy: 0.4050
Epoch 71/100
10/10 [==============================] - 24s 2s/step - loss: 1.3883 - categorical_accuracy: 0.4583 - val_loss: 1.6604 - val_categorical_accuracy: 0.3350
Epoch 72/100
10/10 [==============================] - 26s 3s/step - loss: 1.3297 - categorical_accuracy: 0.4617 - val_loss: 1.5469 - val_categorical_accuracy: 0.4050
Epoch 73/100
10/10 [==============================] - 24s 2s/step - loss: 1.3685 - categorical_accuracy: 0.4433 - val_loss: 1.4889 - val_categorical_accuracy: 0.4550
Epoch 74/100
10/10 [==============================] - 24s 2s/step - loss: 1.4211 - categorical_accuracy: 0.4267 - val_loss: 1.5176 - val_categorical_accuracy: 0.4400
Epoch 75/100
10/1

10/10 [==============================] - 25s 3s/step - loss: 0.9912 - categorical_accuracy: 0.6350 - val_loss: 2.9925 - val_categorical_accuracy: 0.3600
Epoch 68/100
10/10 [==============================] - 24s 2s/step - loss: 0.9416 - categorical_accuracy: 0.6500 - val_loss: 2.6402 - val_categorical_accuracy: 0.3550
Epoch 69/100
10/10 [==============================] - 24s 2s/step - loss: 0.8581 - categorical_accuracy: 0.6800 - val_loss: 2.8459 - val_categorical_accuracy: 0.3400
Epoch 70/100
10/10 [==============================] - 24s 2s/step - loss: 0.9580 - categorical_accuracy: 0.6267 - val_loss: 2.7835 - val_categorical_accuracy: 0.3300
Epoch 71/100
10/10 [==============================] - 23s 2s/step - loss: 0.8264 - categorical_accuracy: 0.6783 - val_loss: 2.8665 - val_categorical_accuracy: 0.3650
Epoch 72/100
10/10 [==============================] - 23s 2s/step - loss: 0.7917 - categorical_accuracy: 0.6917 - val_loss: 2.1538 - val_categorical_accuracy: 0.4400
Epoch 73/100
10/1

Epoch 6/100
10/10 [==============================] - 25s 2s/step - loss: 2.0804 - categorical_accuracy: 0.1883 - val_loss: 2.1936 - val_categorical_accuracy: 0.1900
Epoch 7/100
10/10 [==============================] - 24s 2s/step - loss: 1.9232 - categorical_accuracy: 0.2067 - val_loss: 2.1079 - val_categorical_accuracy: 0.1400
Epoch 8/100
10/10 [==============================] - 23s 2s/step - loss: 1.9223 - categorical_accuracy: 0.1950 - val_loss: 2.0875 - val_categorical_accuracy: 0.1650
Epoch 9/100
10/10 [==============================] - 25s 3s/step - loss: 1.9597 - categorical_accuracy: 0.1917 - val_loss: 2.0988 - val_categorical_accuracy: 0.2150
Epoch 10/100
10/10 [==============================] - 24s 2s/step - loss: 1.8988 - categorical_accuracy: 0.1967 - val_loss: 2.0779 - val_categorical_accuracy: 0.2000
Epoch 11/100
10/10 [==============================] - 23s 2s/step - loss: 1.8639 - categorical_accuracy: 0.1967 - val_loss: 2.1668 - val_categorical_accuracy: 0.2600
Epoch 12